In [1]:
#importing the libraries
import pandas as pd 
from parsel import Selector
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException  # This line is important
import time
import random
import os
import re

In [7]:
from tqdm import tqdm  # Import tqdm for progress bar

# ... existing code ...
# Initialize Chrome options
opts = Options()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")

# Set up the Chrome WebDriver
driver = webdriver.Chrome(options=opts)

# URLs of product categories
urls = [
   "https://www.ajio.com/men-tshirts/c/830216014?query=%3Arelevance&gridColumns=5",
   "https://www.ajio.com/c/830316017?query=%3Arelevance&gridColumns=5"
]

product_types = [
    # "men-sweatshirts",
    "men-tshirts","women-tops"
]

error_links = []
df = pd.DataFrame()  # Initialize DataFrame

# Iterate over URLs with a progress bar
for single_url in tqdm(urls, desc="Processing URLs"):
    driver.get(single_url)
    time.sleep(5)  

    buy_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a.rilrtl-products-list__link') if a.get_attribute('href')]
    buy_links = list(set(buy_links))

    # Iterate over buy links with a progress bar
    for link in tqdm(buy_links, desc="Processing Products", leave=False):
        try:
            driver.get(link)
            time.sleep(5)  # Wait for the page to load

            product_title = driver.find_element(By.CSS_SELECTOR, "h2.brand-name").text  # Updated selector
            product_name = driver.find_element(By.CSS_SELECTOR, "h1.prod-name").text  # Updated selector
            product_price = driver.find_element(By.CSS_SELECTOR, "div.prod-sp").text  # Updated selector for current price
            product_price_disc = driver.find_element(By.CSS_SELECTOR, "span.prod-cp").text  # Updated selector for discounted price

            image_elements = driver.find_elements(By.CSS_SELECTOR, "img.img-alignment")  # Updated selector
            image_urls = [img.get_attribute('src') for img in image_elements]  # Extracting image URLs

            details_elements = driver.find_elements(By.CSS_SELECTOR, "ul.prod-list li.detail-list")  # Updated selector
            product_details = [detail.text for detail in details_elements]  # Extracting product details

            rating_data = driver.find_element(By.CSS_SELECTOR, ".rating-data ._3KsA3")
            overall_rating = rating_data.find_element(By.CSS_SELECTOR, "span._1P7MF").text.split()[0]  # Extract overall rating
            total_ratings = rating_data.find_element(By.CSS_SELECTOR, "div._3AxgC").text  # Extract total ratings
            
            # Store the extracted data
            data = {
                "Product Title": product_title,
                "Product Name": product_name,
                "Product Price": product_price,
                "Product Discounted Price": product_price_disc,
                "Product Details": product_details,
                "Image URLs": image_urls,
                "Product Type": product_types[urls.index(single_url)],  # Match product type with URL
                "Buy Link": link,
                "Overall Rating": overall_rating # Add the buy link to the data
            }

            # Append data to the DataFrame
            df = df.append(data, ignore_index=True)

        except Exception as e:
            error_links.append(link)
            # You can log the error if needed, but no print statement
            # print(f"Error processing link {link}: {e}")

# Save the DataFrame to a CSV file
df.to_csv("extracted_product_ajio.csv", index=False)

# Print error links, if any
if error_links:
    print("Error Links:")
    for error_link in error_links:
        print(error_link)

# Close the driver
driver.quit()

Processing URLs:   0%|          | 0/2 [00:00<?, ?it/s]/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_78437/3582148286.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_78437/3582148286.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_78437/3582148286.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_78437/3582148286.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

Error Links:
https://www.ajio.com/u-s-polo-assn-men-striped-slim-fit-polo-t-shirt/p/469671721_navy
https://www.ajio.com/u-s-polo-assn-logo-print-slim-fit-polo-t-shirt/p/469536305_black
https://www.ajio.com/asos-design-men-crochet-regular-fit-polo-t-shirt/p/469631304_darkorange
https://www.ajio.com/u-s-polo-assn-men-slim-fit-polo-t-shirt/p/469671723_red
https://www.ajio.com/u-s-polo-assn-men-colourblock-slim-fit-polo-t-shirt/p/469671722_navy
https://www.ajio.com/puma-mercedes-amg-petronas-motorsport-regular-fit-crew-neck-t-shirt/p/700191592_black
https://www.ajio.com/s/min-70-percent-off-5539-76311?ref=banner
https://www.ajio.com/u-s-polo-assn-men-striped-slim-fit-polo-t-shirt/p/469671724_black
https://www.ajio.com/puma-brand-print-regular-fit-crew-neck-t-shirt/p/700188438_black
https://www.ajio.com/asos-design-men-pack-of-3-relaxed-fit-crew-neck-t-shirts/p/469631382_multi
https://www.ajio.com/s/min-50-percent-off-5539-43952?ref=banner
https://www.ajio.com/john-players-select-slim-fit-p

In [22]:
import os
import requests
import base64
import pandas as pd

# Configuration
API_KEY = "f85804b527a943f197ee55b7df623528"

headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Function to get the first image URL from the DataFrame
def get_first_image_url(image_urls):
    print(image_urls[0])
    return image_urls[0] if image_urls else ""

# Function to generate description for a product
def generate_description(product_detail, first_image_url):
    # Join the product details into a single string
    product_detail_str = " ".join(product_detail)  # Join list into a single string

    payload = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "You are an AI stylist that helps in generating a description for the product image. Utilize the product specifications to generate an informative description."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": product_detail_str  # Use the concatenated string
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": first_image_url
                        }
                    }
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 300
    }
    ENDPOINT = "https://embedding-model12.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"


    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        a = response.json()
        return a['choices'][0]['message']['content']
    except requests.RequestException as e:
        print(f"Failed to make the request for description. Error: {e}")
        return None

# Assuming df is already defined and populated with product data
for index, row in df.iterrows():
    # Generate description for the current product using product details and image URL
    description = generate_description(row['Product Details'], get_first_image_url(row['Image URLs']))  # Updated to use 'Product Details'
    print(f"Generated description for product at index {index}: {description}")
    # Store the generated description in the DataFrame
    df.at[index, 'Generated Description'] = description

    # Save every 10 requests
    if (index + 1) % 2 == 0:
        df.to_csv("extracted_product_data_with_descriptions.csv", index=False)  # Save up to current index
        print(f"Saved after processing {index + 1} products.")

# Final save after all requests
df.to_csv("extracted_product_data_with_descriptions_ajio.csv", index=False)
print("Final save completed.")

https://assets.ajio.com/medias/sys_master/root/20210828/KMKj/612979f8f997ddce89b036d7/-78Wx98H-460856330-black-MODEL2.jpg
Generated description for product at index 0: Elevate your style and comfort with this PUMA Regular Fit Crew Short Sleeve T-Shirt. Designed for both sports and street-style, this medium-sized tee ensures a relaxed fit for everyday wear. Crafted from soft and breathable cotton, it provides ultimate comfort, making it perfect for any activity. The crew neckline adds a classic touch, while the short sleeves offer a casual look. Easy to care for, this t-shirt is machine washable in warm water. Embrace the world-renowned quality and style of PUMA with this versatile wardrobe staple.

- **Fit**: Regular
- **Sleeve**: Short
- **Size**: Medium
- **Material**: Cotton
- **Care Instructions**: Machine wash warm
- **Product Code**: 460856330001
https://assets.ajio.com/medias/sys_master/root/20240314/xqke/65f2458705ac7d77bbb1c16f/-78Wx98H-466217940-cream-MODEL.jpg
Generated desc

In [24]:
df = df.drop(columns=['Product Details'])
df.to_csv("extracted_product_data_with_descriptions_ajio.csv", index=False)


In [26]:
df['Brand Name'] = 'Ajio'  # Add the brand name
df.to_csv("extracted_product_data_with_descriptions_ajio.csv", index=False)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Set up Chrome options
opts = Options()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")

# Set up the Chrome WebDriver
driver = webdriver.Chrome(options=opts)

# Open the product page
driver.get("https://www.ajio.com/draax-fashions-women-regular-fit-v-neck-top/p/700271978_pink")
time.sleep(5)  # Wait for the page to load

# Extract product details
product_title = driver.find_element(By.CSS_SELECTOR, "h2.brand-name").text  # Updated selector
print("Product Title:", product_title)

product_name = driver.find_element(By.CSS_SELECTOR, "h1.prod-name").text  # Updated selector
print("Product Name:", product_name)
product_price = driver.find_element(By.CSS_SELECTOR, "div.prod-sp").text  
print("Product Price:", product_price)
# Updated selector for current price
product_price_disc = driver.find_element(By.CSS_SELECTOR, "span.prod-cp").text
print("Product Discounted Price:", product_price_disc)
# ... existing code ...
image_elements = driver.find_elements(By.CSS_SELECTOR, "img.img-alignment")  # Updated selector
image_urls = [img.get_attribute('src') for img in image_elements]  # Extracting image URLs
# ... existing code ...
# ... existing code ...
details_elements = driver.find_elements(By.CSS_SELECTOR, "ul.prod-list li.detail-list")  # Updated selector
product_details = [detail.text for detail in details_elements]  # Extracting product details
# ... existing code ...
rating_data = driver.find_element(By.CSS_SELECTOR, ".rating-data ._3KsA3")
overall_rating = rating_data.find_element(By.CSS_SELECTOR, "span._1P7MF").text.split()[0]  # Extract overall rating
total_ratings = rating_data.find_element(By.CSS_SELECTOR, "div._3AxgC").text  # Extract total ratings


print("Image URLs:", image_urls)
print("Product Details:", product_details)
print("Overall Rating:", overall_rating)
print("Total Ratings:", total_ratings)

# Close the driver
driver.quit()

Product Title: DRAAX FASHIONS
Product Name: Women Regular Fit V-Neck Top
Product Price: ₹416
Product Discounted Price: ₹2,449
Image URLs: ['https://assets.ajio.com/medias/sys_master/root/20240808/Alrk/66b4f1fc1d763220fa6b239e/-78Wx98H-700271978-pink-MODEL2.jpg', 'https://assets.ajio.com/medias/sys_master/root/20240808/YsAC/66b4f1fc1d763220fa6b239f/-78Wx98H-700271978-pink-MODEL3.jpg', 'https://assets.ajio.com/medias/sys_master/root/20240808/mGst/66b4f1fc1d763220fa6b23a0/-78Wx98H-700271978-pink-MODEL4.jpg', 'https://assets.ajio.com/medias/sys_master/root/20240808/w9nJ/66b4f1fc1d763220fa6b23a1/-78Wx98H-700271978-pink-MODEL5.jpg', 'https://assets.ajio.com/medias/sys_master/root/20240808/nKNZ/66b4f1fc1d763220fa6b23a2/-78Wx98H-700271978-pink-MODEL6.jpg', 'https://assets.ajio.com/medias/sys_master/root/20240808/1w3t/66b4f1fc1d763220fa6b243c/-78Wx98H-700271978-pink-MODEL.jpg', 'https://assets.ajio.com/medias/sys_master/root/20240808/Alrk/66b4f1fc1d763220fa6b239e/-78Wx98H-700271978-pink-MODEL2.